<a href="https://colab.research.google.com/github/yunjinchoidev/ai/blob/main/kaggle/Natural_Language_Processing_with_Disaster_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 캐글 대회 위치
# https://www.kaggle.com/c/nlp-getting-started

!pip install kaggle
from google.colab import files
files.upload()

In [2]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c nlp-getting-started

  0% 0.00/593k [00:00<?, ?B/s]
100% 593k/593k [00:00<00:00, 81.5MB/s]


In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from numpy import array
from nltk import word_tokenize
from sklearn.model_selection import train_test_split
import re
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn import ensemble

In [6]:
#주어진 문장을 '단어'로 토큰화 하기
import gensim.models
import matplotlib.pyplot as plt
import seaborn as sns

#케라스의 텍스트 전처리와 관련한 함수중 text_to_word_sequence 함수를 불러 옵니다.
from tensorflow.keras.preprocessing.text import text_to_word_sequence
 

In [7]:
!unzip -qq "*.zip"

In [8]:
ls

nlp-getting-started.zip  sample_submission.csv  train.csv
sample_data/             test.csv


In [9]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [10]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [12]:
train.shape

(7613, 5)

In [13]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [14]:
train['keyword'].value_counts()


fatalities               45
deluge                   42
armageddon               42
sinking                  41
damage                   41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: keyword, Length: 221, dtype: int64

In [15]:
print(train.groupby('keyword')['target'].mean().sort_values())


keyword
aftershock     0.000000
body%20bags    0.024390
ruin           0.027027
blazing        0.029412
body%20bag     0.030303
                 ...   
typhoon        0.973684
outbreak       0.975000
debris         1.000000
wreckage       1.000000
derailment     1.000000
Name: target, Length: 221, dtype: float64


In [16]:
print(train.groupby('keyword')['target'].mean())


keyword
ablaze                 0.361111
accident               0.685714
aftershock             0.000000
airplane%20accident    0.857143
ambulance              0.526316
                         ...   
wounded                0.702703
wounds                 0.303030
wreck                  0.189189
wreckage               1.000000
wrecked                0.076923
Name: target, Length: 221, dtype: float64


In [17]:
print('train[\'text\']의 결측치 개수', train['text'].isnull().sum())
print('test[\'text\']의 결측치 개수', test['text'].isnull().sum())
print('train[\'keyword\']의 결측치 개수', train['keyword'].isnull().sum())
print('test[\'keyword\']의 결측치 개수', test['keyword'].isnull().sum())
print('train[\'location\']의 결측치 개수', train['location'].isnull().sum())
print('test[\'location\']의 결측치 개수', test['location'].isnull().sum())

train['text']의 결측치 개수 0
test['text']의 결측치 개수 0
train['keyword']의 결측치 개수 61
test['keyword']의 결측치 개수 26
train['location']의 결측치 개수 2533
test['location']의 결측치 개수 1105


In [18]:
corpus=train['text'].values


In [19]:
corpus=np.concatenate((corpus,test['text'].values), axis=0)


In [20]:
corpus.size


10876

In [21]:
temp=[]
for tem in corpus:
    temp.append(re.sub(r'[^a-zA-Z0-9#@ ]',' ',tem).lower())